# 1. Taiwan Dataset

# 1.1. Taiwan Feature Selection

In [4]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

# Load the dataset
data = pd.read_csv('UCI_Credit_Card.csv')

# Drop the specified features
features_to_drop = ['ID']  # Replace with the features you want to drop
data.drop(columns=features_to_drop, inplace=True)

# Split the dataset into features and target variable
X = data.drop(columns=['default.payment.next.month'])
y = data['default.payment.next.month']

# Anomaly detection using Isolation Forest before scaling
isolation_forest = IsolationForest(contamination=0.1, random_state=42)
outlier_preds = isolation_forest.fit_predict(X)

# Remove outliers
X_cleaned = X[outlier_preds == 1]
y_cleaned = y[outlier_preds == 1]

# Applying Robust Scaling after removing outliers
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_cleaned)

# Perform feature selection using information gain
selector = SelectKBest(score_func=mutual_info_classif, k=5)  # Select top 5 features
X_selected = selector.fit_transform(X_scaled, y_cleaned)

# Display selected feature names
selected_feature_names = X.columns[selector.get_support(indices=True)]
print("Selected Features:", selected_feature_names)

C:\Users\trish\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Selected Features: Index(['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_AMT1'], dtype='object')


# 1.2. Taiwan Main Code

In [5]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from mlxtend.classifier import StackingCVClassifier
from sklearn.tree import DecisionTreeClassifier
from rotation_forest import RotationForestClassifier
from sklearn.ensemble import RandomForestClassifier
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from imblearn.combine import SMOTEENN
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, LSTM
# Load the Taiwan Credit Risk dataset
df = pd.read_csv('UCI_Credit_Card.csv')
df.drop(['PAY_6',  'PAY_5', 'PAY_AMT4', 'PAY_AMT3', 'PAY_AMT6', 'LIMIT_BAL', 'PAY_AMT5', 'PAY_AMT2', 'BILL_AMT1', 'EDUCATION', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT5', 'BILL_AMT6', 'ID', 'BILL_AMT4', 'MARRIAGE', 'AGE', 'SEX'],axis=1, inplace=True)

# Split the dataset into features and target variable
X = df.drop(columns=['default.payment.next.month'])
y = df['default.payment.next.month']

# Rescale data using RobustScaler
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance using SMOTEENN
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_scaled, y)

# Convert data to 3D for LSTM input
X_reshaped = X_resampled.reshape(X_resampled.shape[0], 1, X_resampled.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_resampled, test_size=0.2, random_state=42)

# Define LSTM architecture with multiple layers
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=40, return_sequences=True))
model.add(LSTM(units=30, return_sequences=True))
model.add(LSTM(units=20))  # Last layer doesn't need return_sequences=True
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Print accuracy and loss of each epoch
for epoch, acc in enumerate(history.history['accuracy']):
    print("Epoch", epoch+1, "- Accuracy:", acc, "- Loss:", history.history['loss'][epoch])

# Get predicted probabilities
y_pred_prob = model.predict(X_test)

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate accuracy
overall_accuracy = accuracy_score(y_test, y_pred)
print("Overall Accuracy:", overall_accuracy)

# Calculate F1 score
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

# Calculate AUC score
auc = roc_auc_score(y_test, y_pred_prob)
print("AUC Score:", auc)

Epoch 1/10
380/380 [==============================] - 9s 10ms/step - loss: 0.3690 - accuracy: 0.8687 - val_loss: 0.2793 - val_accuracy: 0.9090
Epoch 2/10
380/380 [==============================] - 3s 7ms/step - loss: 0.2632 - accuracy: 0.9155 - val_loss: 0.2665 - val_accuracy: 0.9117
Epoch 3/10
380/380 [==============================] - 3s 7ms/step - loss: 0.2512 - accuracy: 0.9195 - val_loss: 0.2578 - val_accuracy: 0.9150
Epoch 4/10
380/380 [==============================] - 3s 8ms/step - loss: 0.2453 - accuracy: 0.9211 - val_loss: 0.2604 - val_accuracy: 0.9133
Epoch 5/10
380/380 [==============================] - 3s 7ms/step - loss: 0.2424 - accuracy: 0.9206 - val_loss: 0.2522 - val_accuracy: 0.9133
Epoch 6/10
380/380 [==============================] - 3s 9ms/step - loss: 0.2390 - accuracy: 0.9219 - val_loss: 0.2491 - val_accuracy: 0.9123
Epoch 7/10
380/380 [==============================] - 3s 7ms/step - loss: 0.2362 - accuracy: 0.9210 - val_loss: 0.2490 - val_accuracy: 0.9140
Epoch

# 2. Australian Dataset

# 2.1. Australian Feature Selection

In [9]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

# Load the dataset
df = pd.read_csv('australian.dat', sep=' ', header=None)

# Split the dataset into features and target variable
X = df.drop(columns=[14])
y = df[14]


# Anomaly detection using Isolation Forest before scaling
isolation_forest = IsolationForest(contamination=0.1, random_state=42)
outlier_preds = isolation_forest.fit_predict(X)

# Remove outliers
X_cleaned = X[outlier_preds == 1]
y_cleaned = y[outlier_preds == 1]

# Applying Robust Scaling after removing outliers
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_cleaned)

# Perform feature selection using information gain
selector = SelectKBest(score_func=mutual_info_classif, k=5)  # Select top 5 features
X_selected = selector.fit_transform(X_scaled, y_cleaned)

# Display selected feature names
selected_feature_names = X.columns[selector.get_support(indices=True)]
print("Selected Features:", selected_feature_names)

Selected Features: Int64Index([6, 7, 8, 9, 13], dtype='int64')


# 2.2. Australian Main Code

In [11]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from mlxtend.classifier import StackingCVClassifier
from sklearn.tree import DecisionTreeClassifier
from rotation_forest import RotationForestClassifier
from sklearn.ensemble import RandomForestClassifier
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from imblearn.combine import SMOTEENN
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, LSTM
# Load the dataset
df = pd.read_csv('australian.dat', sep=' ', header=None)
df.drop([0, 1, 2, 3, 4, 5,  10, 11, 12], axis=1, inplace=True)

# Split the dataset into features and target variable
X = df.drop(columns=[14])
y = df[14]

# Rescale data using RobustScaler
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance using SMOTEENN
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_scaled, y)

# Convert data to 3D for LSTM input
X_reshaped = X_resampled.reshape(X_resampled.shape[0], 1, X_resampled.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_resampled, test_size=0.2, random_state=42)

# Define LSTM architecture with multiple layers
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=40, return_sequences=True))
model.add(LSTM(units=30, return_sequences=True))
model.add(LSTM(units=20))  # Last layer doesn't need return_sequences=True
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Print accuracy and loss of each epoch
for epoch, acc in enumerate(history.history['accuracy']):
    print("Epoch", epoch+1, "- Accuracy:", acc, "- Loss:", history.history['loss'][epoch])

# Get predicted probabilities
y_pred_prob = model.predict(X_test)

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate accuracy
overall_accuracy = accuracy_score(y_test, y_pred)
print("Overall Accuracy:", overall_accuracy)

# Calculate F1 score
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

# Calculate AUC score
auc = roc_auc_score(y_test, y_pred_prob)
print("AUC Score:", auc)

Epoch 1/10
10/10 [==============================] - 7s 132ms/step - loss: 0.6929 - accuracy: 0.5286 - val_loss: 0.6922 - val_accuracy: 0.4933
Epoch 2/10
10/10 [==============================] - 0s 11ms/step - loss: 0.6913 - accuracy: 0.5657 - val_loss: 0.6898 - val_accuracy: 0.6800
Epoch 3/10
10/10 [==============================] - 0s 12ms/step - loss: 0.6875 - accuracy: 0.6970 - val_loss: 0.6828 - val_accuracy: 0.8000
Epoch 4/10
10/10 [==============================] - 0s 12ms/step - loss: 0.6772 - accuracy: 0.8451 - val_loss: 0.6633 - val_accuracy: 0.8800
Epoch 5/10
10/10 [==============================] - 0s 13ms/step - loss: 0.6489 - accuracy: 0.9057 - val_loss: 0.6141 - val_accuracy: 0.9067
Epoch 6/10
10/10 [==============================] - 0s 13ms/step - loss: 0.5823 - accuracy: 0.9327 - val_loss: 0.5219 - val_accuracy: 0.9067
Epoch 7/10
10/10 [==============================] - 0s 14ms/step - loss: 0.4811 - accuracy: 0.9293 - val_loss: 0.4340 - val_accuracy: 0.9067
Epoch 8/10
1

# 3. German Dataset

# 3.1. German Feature Selection

In [3]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv('german_credit_data.csv')

# Drop unnecessary columns
df.drop(['Unnamed: 0', 'Checking account'], axis=1, inplace=True)

# Convert categorical variables to numerical form using label encoding
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])
df['Housing'] = le.fit_transform(df['Housing'])
df['Saving accounts'] = le.fit_transform(df['Saving accounts'])
df['Purpose'] = le.fit_transform(df['Purpose'])

# Split the dataset into features and target variable
X = df.drop(columns=['Risk'])
y = df['Risk']

# Anomaly detection using Isolation Forest before scaling
isolation_forest = IsolationForest(contamination=0.1, random_state=42)
outlier_preds = isolation_forest.fit_predict(X)

# Remove outliers
X_cleaned = X[outlier_preds == 1]
y_cleaned = y[outlier_preds == 1]

# Applying Robust Scaling after removing outliers
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_cleaned)

# Perform feature selection using information gain
selector = SelectKBest(score_func=mutual_info_classif, k=5)  # Select top 5 features
X_selected = selector.fit_transform(X_scaled, y_cleaned)

# Display selected feature names
selected_feature_names = X.columns[selector.get_support(indices=True)]
print("Selected Features:", selected_feature_names)

Selected Features: Index(['Age', 'Sex', 'Saving accounts', 'Duration', 'Purpose'], dtype='object')


C:\Users\trish\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


# 3.2. German Main Code

In [5]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from imblearn.combine import SMOTEENN
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import LabelEncoder

# Load the German Credit Risk dataset
df = pd.read_csv('german_credit_data.csv')

# Drop unnecessary columns
df.drop(['Unnamed: 0', 'Job','Housing','Checking account','Credit amount', 'Risk'], axis=1, inplace=True)

# Convert categorical variables to numerical form using label encoding
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])
#df['Housing'] = le.fit_transform(df['Housing'])
#df['Risk'] = le.fit_transform(df['Risk'])
df['Saving accounts'] = le.fit_transform(df['Saving accounts'])
#df['Checking account'] = le.fit_transform(df['Checking account'])
df['Purpose'] = le.fit_transform(df['Purpose'])

# Split the dataset into input and output variables
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Rescale data using RobustScaler
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance using SMOTEENN
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_scaled, y)

# Convert data to 3D for LSTM input
X_reshaped = X_resampled.reshape(X_resampled.shape[0], 1, X_resampled.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_resampled, test_size=0.2, random_state=42)

# Define LSTM architecture with multiple layers
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=40, return_sequences=True))
model.add(LSTM(units=30, return_sequences=True))
model.add(LSTM(units=20))  # Last layer doesn't need return_sequences=True
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Print accuracy and loss of each epoch
for epoch, acc in enumerate(history.history['accuracy']):
    print("Epoch", epoch+1, "- Accuracy:", acc, "- Loss:", history.history['loss'][epoch])

# Get predicted probabilities
y_pred_prob = model.predict(X_test)

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Convert y_test to binary labels
y_test_binary = (y_test > 0).astype(int)

# Calculate accuracy
overall_accuracy = accuracy_score(y_test_binary, y_pred)
print("Overall Accuracy:", overall_accuracy)

# Calculate F1 score
f1 = f1_score(y_test_binary, y_pred)
print("F1 Score:", f1)

# Calculate AUC score
auc = roc_auc_score(y_test_binary, y_pred_prob)
print("AUC Score:", auc)

Epoch 1/10
16/16 [==============================] - 8s 85ms/step - loss: 0.6122 - accuracy: 0.0098 - val_loss: 0.5238 - val_accuracy: 0.0078
Epoch 2/10
16/16 [==============================] - 0s 9ms/step - loss: 0.3813 - accuracy: 0.0039 - val_loss: 0.2382 - val_accuracy: 0.0078
Epoch 3/10
16/16 [==============================] - 0s 10ms/step - loss: -0.1226 - accuracy: 0.0039 - val_loss: -0.5215 - val_accuracy: 0.0078
Epoch 4/10
16/16 [==============================] - 0s 10ms/step - loss: -1.6974 - accuracy: 0.0039 - val_loss: -3.1478 - val_accuracy: 0.0078
Epoch 5/10
16/16 [==============================] - 0s 11ms/step - loss: -6.3938 - accuracy: 0.0039 - val_loss: -8.4024 - val_accuracy: 0.0078
Epoch 6/10
16/16 [==============================] - 0s 10ms/step - loss: -12.5408 - accuracy: 0.0039 - val_loss: -13.2666 - val_accuracy: 0.0078
Epoch 7/10
16/16 [==============================] - 0s 10ms/step - loss: -17.2283 - accuracy: 0.0039 - val_loss: -16.7307 - val_accuracy: 0.0078
